In [ ]:
# real_time_ai_mood_buddy.py

import cv2
import numpy as np
import torch
import torch.nn.functional as F
import librosa
import sounddevice as sd
import pyttsx3
from collections import Counter, deque
from tensorflow.keras.models import load_model
from utils.preprocessing import preprocess_input

# ========== CONFIG ==========
FACE_MODEL_PATH = "fer2013_mini_XCEPTION.119-0.65.hdf5"
VOICE_MODEL_PATH = "emotion_detection/voice_model_traced.pt"
EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
COLOR_MAP = {
    'Angry': (0, 0, 255), 'Disgust': (0, 128, 0), 'Fear': (128, 0, 128),
    'Happy': (0, 255, 255), 'Sad': (255, 0, 0), 'Surprise': (255, 165, 0), 'Neutral': (128, 128, 128)
}

def get_chatbot_response(emotion):
    responses = {
        'Happy': "You look happy! Keep smiling!",
        'Sad': "I'm here for you. Want to hear a story or a joke?",
        'Angry': "Take a deep breath. It's okay to feel angry sometimes.",
        'Fear': "Don't worry, I'm with you. Want to play a game to feel better?",
        'Disgust': "Let's find something fun to do instead!",
        'Surprise': "Wow! Something exciting must've happened!",
        'Neutral': "Hope you're having a calm and peaceful moment."
    }
    return responses.get(emotion, "Tell me more about how you're feeling.")

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def predict_face_emotion(face_img, model):
    face_img = cv2.resize(face_img, (48, 48))
    face_img = preprocess_input(face_img)
    face_img = np.expand_dims(face_img, axis=0)[..., np.newaxis]
    preds = model.predict(face_img, verbose=0)
    return EMOTION_LABELS[np.argmax(preds)]

def predict_voice_emotion(audio, model):
    mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    x = np.mean(mfcc.T, axis=0)
    x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        out = model(x_tensor)
        probs = F.softmax(out, dim=1)
    return EMOTION_LABELS[torch.argmax(probs)]

def record_voice(duration=3, sr=16000):
    print("🎙️ Recording voice... Speak now")
    audio = sd.rec(int(sr * duration), samplerate=sr, channels=1, dtype='float32')
    sd.wait()
    return audio.flatten()

# ====== INIT ======
face_model = load_model(FACE_MODEL_PATH, compile=False)
voice_model = torch.load(VOICE_MODEL_PATH, map_location=torch.device('cpu'),weights_only=False)
voice_model.eval()
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

emotion_window = deque(maxlen=10)
stable_emotion = None

# ====== VIDEO LOOP ======
cap = cv2.VideoCapture(0)
print("\n🟢 AI Mood Buddy Started - Press 'v' to use voice | 'q' to quit\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_emotion = predict_face_emotion(face, face_model)
        emotion_window.append(face_emotion)

        common_emotion, count = Counter(emotion_window).most_common(1)[0]
        box_color = COLOR_MAP.get(common_emotion, (255, 255, 255))

        cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
        cv2.putText(frame, common_emotion, (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

        if common_emotion != stable_emotion and count >= 7:
            stable_emotion = common_emotion
            response = get_chatbot_response(common_emotion)
            print(f"\n😀 Detected Emotion: {common_emotion}\n🤖 {response}\n")
            speak(response)

    cv2.imshow("AI Mood Buddy", frame)
    key = cv2.waitKey(1)

    if key == ord('v'):
        audio = record_voice()
        voice_emotion = predict_voice_emotion(audio, voice_model)
        response = get_chatbot_response(voice_emotion)
        print(f"\n🗣️ Detected Voice Emotion: {voice_emotion}\n🤖 {response}\n")
        speak(response)
        stable_emotion = voice_emotion  # override face temporarily

    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



🟢 AI Mood Buddy Started - Press 'v' to use voice | 'q' to quit


😀 Detected Emotion: Sad
🤖 I'm here for you. Want to hear a story or a joke?


😀 Detected Emotion: Neutral
🤖 Hope you're having a calm and peaceful moment.


😀 Detected Emotion: Happy
🤖 You look happy! Keep smiling!


😀 Detected Emotion: Neutral
🤖 Hope you're having a calm and peaceful moment.


😀 Detected Emotion: Surprise
🤖 Wow! Something exciting must've happened!


😀 Detected Emotion: Happy
🤖 You look happy! Keep smiling!


😀 Detected Emotion: Fear
🤖 Don't worry, I'm with you. Want to play a game to feel better?


😀 Detected Emotion: Neutral
🤖 Hope you're having a calm and peaceful moment.


😀 Detected Emotion: Sad
🤖 I'm here for you. Want to hear a story or a joke?


😀 Detected Emotion: Neutral
🤖 Hope you're having a calm and peaceful moment.


😀 Detected Emotion: Happy
🤖 You look happy! Keep smiling!


😀 Detected Emotion: Neutral
🤖 Hope you're having a calm and peaceful moment.


😀 Detected Emotion: Happy
🤖 You look